In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv
%matplotlib inline

from os.path import join

import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.utils import to_categorical

In [2]:
rows = []
drugFeatures = {}

with open('drugFeature.csv', 'r') as csvfile: 
    reader = csv.reader(csvfile, skipinitialspace=True)
    next(reader)
    for row in reader:
        rows.append(row)
        
for i in range(0, len(rows)):
    drugFeatures[rows[i][0]] = rows[i][1:]

In [3]:
rows = []
miRNAFeatures = {}

with open('miRNAFeature.csv', 'r') as csvfile: 
    reader = csv.reader(csvfile, skipinitialspace=True)
    next(reader)
    for row in reader:
        rows.append(row)
        
for i in range(0, len(rows)):
    miRNAFeatures[rows[i][0]] = rows[i][1:]


In [4]:
miRNA_train = []
drug_train = []
correlation_train = []

with open('correlationData.csv', 'r') as csvfile: 
    reader = csv.reader(csvfile, skipinitialspace=True)
    next(reader)
    for miRBase, cid, correlation in reader:
        drug_train.append(drugFeatures[str(cid)])
        correlation_train.append(correlation)
        miRNA_train.append(miRNAFeatures[str(miRBase)])

for i in range(0, len(miRNA_train)):
    miRNA_train[i] = miRNA_train[i] + drug_train[i]

#print(miRNA_train[0])
    
train_x = np.array(miRNA_train)
print(train_x.shape)

#drug_train = np.array(drug_train)
#print(drug_train.shape)

#miRNA_train = np.array(miRNA_train)
#print(miRNA_train.shape)

correlation_train = np.array(correlation_train)
correlation_train_onehot = to_categorical(correlation_train)
print(correlation_train_onehot.shape)
correlation_train_onehot[5000]
train_x[0]

(6993, 256)
(6993, 2)


array(['0.998213828', '0', '0.196423188', '1.236059785', '0.387064487',
       '0.620136678', '0.692301929', '1.033843398', '0', '0',
       '0.601664245', '0.148767635', '0.274716407', '0.431130171',
       '0.419233263', '0.845790207', '0', '0.312125385', '0.749852121',
       '0', '0.634588659', '0.806407809', '0.722176433', '1.136551142',
       '0.850761056', '0', '0.815267384', '1.730976582', '0.371746927',
       '1.700378895', '1.164160371', '0', '0', '0', '0', '0.852529466',
       '0.355800867', '0', '0', '0.37191686', '1.029871225',
       '1.512212873', '0', '0.942647934', '0.637958527', '0.177883118',
       '0.21051386', '0.245826408', '0.228340894', '0.219275415', '0',
       '0.906445622', '0.849691272', '0.779798329', '1.504798293', '0',
       '1.400229335', '0.910516024', '0.67655915', '0.725000918',
       '0.670768261', '0.838796735', '0.903507948', '0.570242941',
       '1.583002925', '1.451573133', '0.760110974', '1.003131151',
       '0.615719318', '0.778978884'

In [5]:
#miRNAFeatureInput = layers.Input(shape=(128,))
#drugFeatureInput = layers.Input(shape=(128,))
Input_layer = layers.Input(shape=(256,))
layer1 = layers.Dense(units=1000, activation = 'relu')(Input_layer)
layer2 = layers.Dense(units=1000, activation = 'relu')(layer1)
layer3 = layers.Dense(units=2000, activation = 'relu')(layer2)
layer4 = layers.Dense(units=1000, activation = 'relu')(layer3)
output = layers.Dense(units=2, activation = 'softmax')(layer4)

In [6]:
predict_model = models.Model(Input_layer, output)

In [7]:
predict_model.compile(optimizer='Adam', loss="categorical_crossentropy")

In [8]:
predict_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 256)]             0         
_________________________________________________________________
dense (Dense)                (None, 1000)              257000    
_________________________________________________________________
dense_1 (Dense)              (None, 1000)              1001000   
_________________________________________________________________
dense_2 (Dense)              (None, 2000)              2002000   
_________________________________________________________________
dense_3 (Dense)              (None, 1000)              2001000   
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 2002      
Total params: 5,263,002
Trainable params: 5,263,002
Non-trainable params: 0
___________________________________________________

In [ ]:
predict_model.fit(train_x, correlation_train_onehot, verbose = 1, validation_split = 0.2, epochs = 1, batch_size = 32)